90天波動係數

In [12]:
import pandas as pd
import numpy as np
df = pd.read_csv('50合併.csv')
# 假设df是您的DataFrame
# 将日期转换为datetime格式
df['Date'] = pd.to_datetime(df['Date'])

# 将'Close'列转换为数值型
df['Adj_Close'] = pd.to_numeric(df['Adj_Close'], errors='coerce')

# 删除包含NaN的行，或者选择适合您的数据处理方式
df = df.dropna(subset=['Adj_Close'])

# 按照'ticker'和'Date'排序
df.sort_values(['Ticker', 'Date'], inplace=True)

# 对每个'ticker'进行滚动计算
df['90_day_std'] = df.groupby('Ticker')['Adj_Close'].rolling(window=90).std().reset_index(level=0, drop=True)
df['90_day_avg'] = df.groupby('Ticker')['Adj_Close'].rolling(window=90).mean().reset_index(level=0, drop=True)

# 计算90天波动系数
df['90DCV'] = df['90_day_std'] / df['90_day_avg']

# 输出结果
print(df[['Date', 'Ticker', 'Adj_Close', '90_day_std', '90_day_avg', '90DCV']].tail())


            Date  Ticker  Adj_Close  90_day_std  90_day_avg     90DCV
93787 2024-10-04    6505       54.5    5.665928   60.189778  0.094134
93788 2024-10-07    6505       55.0    5.655180   60.065111  0.094151
93789 2024-10-08    6505       54.2    5.655891   59.936889  0.094364
93790 2024-10-09    6505       52.3    5.660582   59.772556  0.094702
93791 2024-10-11    6505       52.0    5.678327   59.615667  0.095249


In [14]:
# 选择要导出的列
export_columns = ['Date', 'Ticker', '90DCV']

# 导出到 CSV 文件
df[export_columns].to_csv('90DCV_output.csv', index=False)

Beta係數

In [24]:
import pandas as pd
import numpy as np

# 1. 数据加载
df_stocks = pd.read_csv('50合併.csv')

# 2. 数据预处理

# 转换 'Date' 列为 datetime 格式
df_stocks['Date'] = pd.to_datetime(df_stocks['Date'], format='%Y/%m/%d')

# 确保 'ticker' 列为字符串类型
df_stocks['ticker'] = df_stocks['ticker'].astype(str)

# 转换 'Adj_Close' 列为数值型，无法转换的值设为 NaN
df_stocks['Adj_Close'] = pd.to_numeric(df_stocks['Adj_Close'], errors='coerce')

# 按照 'ticker' 和 'Date' 排序
df_stocks.sort_values(['ticker', 'Date'], inplace=True)

# 计算每日的对数回报率，使用 transform 保持索引对齐
df_stocks['Log_Return'] = df_stocks.groupby('ticker')['Adj_Close'].transform(
    lambda x: np.log(x / x.shift(1))
)

# 定义滚动窗口的大小（21 天）
window_size = 21

# 计算已实现波动率（每个 ticker 分组计算），使用 transform 保持索引对齐
df_stocks['Realized_Vol'] = df_stocks.groupby('ticker')['Log_Return'].transform(
    lambda x: np.sqrt(252) * np.sqrt((x**2).rolling(window=window_size).mean())
)

# 移除包含 NaN 的行（由于滚动窗口导致的前 window-1 天数据缺失）
df_stocks = df_stocks.dropna(subset=['Realized_Vol'])

# 3. 导出结果

# 选择要导出的列
export_columns = ['Date', 'ticker', 'Adj_Close', 'Log_Return', 'Realized_Vol']

# 检查 export_columns 中的列是否存在于 df_stocks 中
missing_cols = [col for col in export_columns if col not in df_stocks.columns]
if missing_cols:
    print(f"以下列不存在于 DataFrame 中，无法导出: {missing_cols}")
else:
    # 导出为 CSV 文件
    df_stocks[export_columns].to_csv('Realized_Vol_results.csv', index=False)
    print("已实现波动率结果已成功导出到 'Realized_Vol_results.csv'。")
    
    # 如果需要导出为 Excel 文件，请取消以下注释
    # df_stocks[export_columns].to_excel('Realized_Vol_results.xlsx', index=False)
    # print("已实现波动率结果已成功导出到 'Realized_Vol_results.xlsx'。")


已实现波动率结果已成功导出到 'Realized_Vol_results.csv'。


一個月實現波動率

In [91]:
import pandas as pd
import numpy as np

# Load the uploaded CSV file
file_path = '50合併.csv'
df = pd.read_csv(file_path)

# Convert Date column to datetime format and sort by date and ticker
df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')
df = df.sort_values(by=['ticker', 'Date'])

# Convert relevant columns to numeric types to avoid type errors
df['Adj_Close'] = pd.to_numeric(df['Adj_Close'], errors='coerce')

# Ensure each group has a continuous business day index
df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()

# Calculate daily log returns using a loop to avoid index issues
log_return_list = []
for name, group in df.groupby('ticker'):
    group['log_return'] = np.log(group['Adj_Close'] / group['Adj_Close'].shift(1))
    log_return_list.append(group)

df = pd.concat(log_return_list)

# Calculate realized annualized volatility for the past month (approx. 21 trading days)
vol_list = []
for name, group in df.groupby('ticker'):
    group['AnnVol1M'] = group['log_return'].rolling(window=21).apply(lambda x: np.sqrt(252 * np.sum(x**2) / len(x)))
    vol_list.append(group)

df = pd.concat(vol_list)

# Filter the DataFrame to remove rows with missing values in the calculated columns for AnnVol1M
df_filtered_annvol1m = df[['Date', 'ticker', 'Adj_Close', 'log_return', 'AnnVol1M']].dropna()

# Display the filtered results for AnnVol1M
print(df_filtered_annvol1m)

C:\Users\asas9\AppData\Local\Temp\ipykernel_28296\3511032184.py:6: DtypeWarning: Columns (2,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\asas9\AppData\Local\Temp\ipykernel_28296\3511032184.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()


             Date  ticker  Adj_Close  log_return  AnnVol1M
21     2016-11-09  1101.0      18.92   -0.032244  0.263921
22     2016-11-10  1101.0      19.88    0.049495  0.314723
23     2016-11-11  1101.0      19.54   -0.017251  0.303366
24     2016-11-14  1101.0      19.31   -0.011841  0.294780
25     2016-11-15  1101.0      19.13   -0.009365  0.295862
...           ...     ...        ...         ...       ...
100266 2024-10-04  6505.0      54.50    0.041204  0.408853
100267 2024-10-07  6505.0      55.00    0.009132  0.409639
100268 2024-10-08  6505.0      54.20   -0.014652  0.356282
100269 2024-10-09  6505.0      52.30   -0.035685  0.366612
100271 2024-10-11  6505.0      52.00   -0.005753  0.364849

[92401 rows x 5 columns]


In [93]:
export_columns = ['Date', 'ticker', 'AnnVol1M']
# 导出到 CSV 文件
df_filtered_annvol1m[export_columns].to_csv('AnnVol1M.csv', index=False)

12個月年化波動率

In [95]:
import pandas as pd
import numpy as np

# Load the uploaded CSV file
file_path = '50合併.csv'
df = pd.read_csv(file_path)

# Convert Date column to datetime format and sort by date and ticker
df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')
df = df.sort_values(by=['ticker', 'Date'])

# Convert relevant columns to numeric types to avoid type errors
df['Adj_Close'] = pd.to_numeric(df['Adj_Close'], errors='coerce')

# Ensure each group has a continuous business day index
df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()

# Calculate daily log returns using a loop to avoid index issues
log_return_list = []
for name, group in df.groupby('ticker'):
    group['log_return'] = np.log(group['Adj_Close'] / group['Adj_Close'].shift(1))
    log_return_list.append(group)

df = pd.concat(log_return_list)

# Calculate realized annualized volatility for the past year (approx. 252 trading days)
df['AnnVol12M'] = df.groupby('ticker')['log_return'].rolling(window=252).apply(lambda x: np.sqrt(252 * np.sum(x**2) / len(x))).reset_index(level=0, drop=True)

# Filter the DataFrame to remove rows with missing values in the calculated columns for AnnVol12M
df_filtered_annvol12m = df[['Date', 'ticker', 'Adj_Close', 'log_return', 'AnnVol12M']].dropna()

# Display the filtered results for AnnVol12M
print(df_filtered_annvol12m)

C:\Users\asas9\AppData\Local\Temp\ipykernel_28296\804330945.py:6: DtypeWarning: Columns (2,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\asas9\AppData\Local\Temp\ipykernel_28296\804330945.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()


             Date  ticker  Adj_Close  log_return  AnnVol12M
269    2017-10-23  1101.0      18.35   -0.003264   0.178680
270    2017-10-24  1101.0      18.22   -0.007110   0.178821
271    2017-10-25  1101.0      18.19   -0.001648   0.176344
272    2017-10-26  1101.0      18.16   -0.001651   0.174733
273    2017-10-27  1101.0      18.16    0.000000   0.174635
...           ...     ...        ...         ...        ...
100266 2024-10-04  6505.0      54.50    0.041204   0.229437
100267 2024-10-07  6505.0      55.00    0.009132   0.229590
100268 2024-10-08  6505.0      54.20   -0.014652   0.229299
100269 2024-10-09  6505.0      52.30   -0.035685   0.232027
100271 2024-10-11  6505.0      52.00   -0.005753   0.232086

[80336 rows x 5 columns]


In [97]:
export_columns = ['Date', 'ticker', 'AnnVol12M']
# 导出到 CSV 文件
df_filtered_annvol12m[export_columns].to_csv('AnnVol12M.csv', index=False)

銷售對企業價值比

In [6]:
import pandas as pd

# Load the uploaded CSV file
file_path = '50合併.csv'
df = pd.read_csv(file_path)

# Convert Date column to datetime format and sort by date and ticker
df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')
df = df.sort_values(by=['ticker', 'Date'])

# Convert relevant columns to numeric types to avoid type errors
df['SALEQ'] = pd.to_numeric(df['SALEQ'], errors='coerce')
df['CSHOQ'] = pd.to_numeric(df['CSHOQ'], errors='coerce')
df['Adj_Close'] = pd.to_numeric(df['Adj_Close'], errors='coerce')
df['DLTTQ'] = pd.to_numeric(df['DLTTQ'], errors='coerce')
df['DLCQ'] = pd.to_numeric(df['DLCQ'], errors='coerce')
df['PSTKQ'] = pd.to_numeric(df['PSTKQ'], errors='coerce')
df['MIBTQ'] = pd.to_numeric(df['MIBTQ'], errors='coerce')
df['CHEQ'] = pd.to_numeric(df['CHEQ'], errors='coerce')

# Remove rows with missing values initially to ensure alignment is correct
df_cleaned = df.dropna()

# Set Date as the index and ensure continuity for each ticker by reindexing to include all business days
df_cleaned = df_cleaned.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()

# Recalculate rolling metrics using a rolling window of approximately 252 trading days (to approximate 4 quarters), with min_periods set to 252
df_cleaned['Total_Sales_4_Quarters_Rolling'] = df_cleaned.groupby('ticker')['SALEQ'].rolling(window=252, min_periods=252).sum().reset_index(level=0, drop=True)
df_cleaned['Avg_CSHOQ_4_Quarters_Rolling'] = df_cleaned.groupby('ticker')['CSHOQ'].rolling(window=252, min_periods=252).mean().reset_index(level=0, drop=True)

# Recalculate the enterprise value (EV) using rolling 4-quarter data, rolling daily
df_cleaned['EV_4_Quarters_Rolling'] = (df_cleaned['Avg_CSHOQ_4_Quarters_Rolling'] * df_cleaned['Adj_Close']) + df_cleaned['DLTTQ'] + df_cleaned['DLCQ'] + df_cleaned['PSTKQ'] + df_cleaned['MIBTQ'] - df_cleaned['CHEQ']

# Recalculate the sales-to-enterprise value ratio (SEV) using rolling 4-quarter data, rolling daily
df_cleaned['SEV_4_Quarters_Rolling'] = df_cleaned['Total_Sales_4_Quarters_Rolling'] / df_cleaned['EV_4_Quarters_Rolling']

# Filter the DataFrame to remove rows with missing values in the calculated columns
df_filtered_cleaned = df_cleaned[['Date', 'ticker', 'Total_Sales_4_Quarters_Rolling', 'EV_4_Quarters_Rolling', 'SEV_4_Quarters_Rolling']].dropna()


C:\Users\asas9\AppData\Local\Temp\ipykernel_7732\3647532618.py:5: DtypeWarning: Columns (2,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\asas9\AppData\Local\Temp\ipykernel_7732\3647532618.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_cleaned = df_cleaned.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()


In [8]:
df_filtered_cleaned

,Date,ticker,Total_Sales_4_Quarters_Rolling,EV_4_Quarters_Rolling,SEV_4_Quarters_Rolling
268,2017-10-20,1101.0,5.801347e+12,1.984236e+11,29.237173
269,2017-10-23,1101.0,5.803119e+12,1.982021e+11,29.278792
270,2017-10-24,1101.0,5.804890e+12,1.977221e+11,29.358830
271,2017-10-25,1101.0,5.806662e+12,1.976114e+11,29.384253
272,2017-10-26,1101.0,5.808434e+12,1.975006e+11,29.409704
...,...,...,...,...,...
100266,2024-10-04,6505.0,4.476381e+13,5.958281e+11,75.128734
100267,2024-10-07,6505.0,4.477503e+13,6.005911e+11,74.551607
100268,2024-10-08,6505.0,4.478625e+13,5.929703e+11,75.528653
100269,2024-10-09,6505.0,4.479747e+13,5.748710e+11,77.926125


In [10]:
export_columns = ['Date', 'ticker', 'SEV_4_Quarters_Rolling']
# 导出到 CSV 文件
df_filtered_cleaned[export_columns].to_csv('SEV.csv', index=False)

****股息殖利率

In [27]:
import pandas as pd

# Load the uploaded CSV file
file_path = '50合併.csv'
df = pd.read_csv(file_path)

# Convert Date column to datetime format and sort by date and ticker
df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')
df = df.sort_values(by=['ticker', 'Date'])

# Convert relevant columns to numeric types to avoid type errors
df['DVQ'] = pd.to_numeric(df['DVQ'], errors='coerce')
df['CSHOQ'] = pd.to_numeric(df['CSHOQ'], errors='coerce')
df['Adj_Close'] = pd.to_numeric(df['Adj_Close'], errors='coerce')

# Ensure each group has a continuous business day index
df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()

# Calculate rolling sum of DVQ over the past 4 quarters (approx. 252 trading days)
df['Total_DVQ_4_Quarters_Rolling'] = df.groupby('ticker')['DVQ'].rolling(window=252, min_periods=252).sum().reset_index(level=0, drop=True)

# Calculate rolling average of CSHOQ over the past 4 quarters (approx. 252 trading days)
df['Avg_CSHOQ_4_Quarters_Rolling'] = df.groupby('ticker')['CSHOQ'].rolling(window=252, min_periods=252).mean().reset_index(level=0, drop=True)

# Calculate dividend yield (DivP) based on the provided formula
df['DivP'] = df['Total_DVQ_4_Quarters_Rolling'] / (df['Adj_Close'] * df['Avg_CSHOQ_4_Quarters_Rolling'])

# Filter the DataFrame to remove rows with missing values in the calculated columns
df_filtered = df[['Date', 'ticker', 'Total_DVQ_4_Quarters_Rolling', 'Avg_CSHOQ_4_Quarters_Rolling', 'Adj_Close', 'DivP']].dropna()

# Display the filtered results
print(df_filtered)

             Date  ticker  Total_DVQ_4_Quarters_Rolling  \
268    2017-10-20  1101.0                           0.0   
269    2017-10-23  1101.0                           0.0   
270    2017-10-24  1101.0                           0.0   
271    2017-10-25  1101.0                           0.0   
272    2017-10-26  1101.0                           0.0   
...           ...     ...                           ...   
100266 2024-10-04  6505.0                           0.0   
100267 2024-10-07  6505.0                           0.0   
100268 2024-10-08  6505.0                           0.0   
100269 2024-10-09  6505.0                           0.0   
100271 2024-10-11  6505.0                           0.0   

        Avg_CSHOQ_4_Quarters_Rolling  Adj_Close  DivP  
268                     3.692176e+09      18.41   0.0  
269                     3.692176e+09      18.35   0.0  
270                     3.692176e+09      18.22   0.0  
271                     3.692176e+09      18.19   0.0  
272        

C:\Users\asas9\AppData\Local\Temp\ipykernel_28296\2905873931.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()


In [31]:
export_columns = ['Date', 'ticker', 'DivP']
# 导出到 CSV 文件
df_filtered[export_columns].to_csv('DivP.csv', index=False)

In [ ]:
每股收益殖利率

In [41]:
import pandas as pd

# Load the uploaded CSV file
file_path = '50合併.csv'
df = pd.read_csv(file_path)

# Convert Date column to datetime format and sort by date and ticker
df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')
df = df.sort_values(by=['ticker', 'Date'])

# Convert relevant columns to numeric types to avoid type errors
df['EPS'] = pd.to_numeric(df['EPS'], errors='coerce')
df['Adj_Close'] = pd.to_numeric(df['Adj_Close'], errors='coerce')

# Ensure each group has a continuous business day index
df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()

# Calculate rolling sum of EPS over the past 4 quarters (approx. 252 trading days)
df['Total_EPS_4_Quarters_Rolling'] = df.groupby('ticker')['EPS'].rolling(window=252, min_periods=252).sum().reset_index(level=0, drop=True)

# Calculate earnings yield (EP) based on the provided formula
df['EP'] = df['Total_EPS_4_Quarters_Rolling'] / df['Adj_Close']

# Filter the DataFrame to remove rows with missing values in the calculated columns
df_filtered = df[['Date', 'ticker', 'Total_EPS_4_Quarters_Rolling', 'Adj_Close', 'EP']].dropna()

# Display the filtered results
print(df_filtered)


C:\Users\asas9\AppData\Local\Temp\ipykernel_28296\2656754767.py:5: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


             Date  ticker  Total_EPS_4_Quarters_Rolling  Adj_Close        EP
268    2017-10-20  1101.0                        125.13      18.41  6.796850
269    2017-10-23  1101.0                        125.17      18.35  6.821253
270    2017-10-24  1101.0                        125.21      18.22  6.872119
271    2017-10-25  1101.0                        125.25      18.19  6.885651
272    2017-10-26  1101.0                        125.29      18.16  6.899229
...           ...     ...                           ...        ...       ...
100266 2024-10-04  6505.0                        205.04      54.50  3.762202
100267 2024-10-07  6505.0                        205.45      55.00  3.735455
100268 2024-10-08  6505.0                        205.86      54.20  3.798155
100269 2024-10-09  6505.0                        206.27      52.30  3.943977
100271 2024-10-11  6505.0                        206.68      52.00  3.974615

[80116 rows x 5 columns]


C:\Users\asas9\AppData\Local\Temp\ipykernel_28296\2656754767.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()


In [43]:
export_columns = ['Date', 'ticker', 'EP']
# 导出到 CSV 文件
df_filtered[export_columns].to_csv('EP.csv', index=False)

EBITDA對企業價值比

In [45]:
import pandas as pd

# Load the uploaded CSV file
file_path = '50合併.csv'
df = pd.read_csv(file_path)

# Convert Date column to datetime format and sort by date and ticker
df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')
df = df.sort_values(by=['ticker', 'Date'])

# Convert relevant columns to numeric types to avoid type errors
df['SALEQ'] = pd.to_numeric(df['SALEQ'], errors='coerce')
df['CSHOQ'] = pd.to_numeric(df['CSHOQ'], errors='coerce')
df['Adj_Close'] = pd.to_numeric(df['Adj_Close'], errors='coerce')
df['DLTTQ'] = pd.to_numeric(df['DLTTQ'], errors='coerce')
df['DLCQ'] = pd.to_numeric(df['DLCQ'], errors='coerce')
df['PSTKQ'] = pd.to_numeric(df['PSTKQ'], errors='coerce')
df['MIBTQ'] = pd.to_numeric(df['MIBTQ'], errors='coerce')
df['CHEQ'] = pd.to_numeric(df['CHEQ'], errors='coerce')

# Ensure each group has a continuous business day index
df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()

# Calculate Enterprise Value (EV) based on the given formula
df['EV'] = (df['CSHOQ'] * df['Adj_Close']) + df['DLTTQ'] + df['DLCQ'] + df['PSTKQ'] + df['MIBTQ'] - df['CHEQ']

# Calculate rolling sum of SALEQ over the past 4 quarters (approx. 252 trading days)
df['Total_SALEQ_4_Quarters_Rolling'] = df.groupby('ticker')['SALEQ'].rolling(window=252, min_periods=252).sum().reset_index(level=0, drop=True)

# Calculate rolling average of EV over the past 4 quarters (approx. 252 trading days)
df['Avg_EV_4_Quarters_Rolling'] = df.groupby('ticker')['EV'].rolling(window=252, min_periods=252).mean().reset_index(level=0, drop=True)

# Calculate EBITDA-to-EV ratio (EBITDAEV) based on the provided formula
df['EBITDAEV'] = df['Total_SALEQ_4_Quarters_Rolling'] / df['Avg_EV_4_Quarters_Rolling']

# Filter the DataFrame to remove rows with missing values in the calculated columns
df_filtered = df[['Date', 'ticker', 'Total_SALEQ_4_Quarters_Rolling', 'Avg_EV_4_Quarters_Rolling', 'EBITDAEV']].dropna()

# Display the filtered results
print(df_filtered)

C:\Users\asas9\AppData\Local\Temp\ipykernel_28296\366518415.py:5: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


             Date  ticker  Total_SALEQ_4_Quarters_Rolling  \
268    2017-10-20  1101.0                    2.018721e+12   
269    2017-10-23  1101.0                    2.018847e+12   
270    2017-10-24  1101.0                    2.018973e+12   
271    2017-10-25  1101.0                    2.019099e+12   
272    2017-10-26  1101.0                    2.019225e+12   
...           ...     ...                             ...   
100266 2024-10-04  6505.0                    1.469524e+13   
100267 2024-10-07  6505.0                    1.467818e+13   
100268 2024-10-08  6505.0                    1.466113e+13   
100269 2024-10-09  6505.0                    1.464408e+13   
100271 2024-10-11  6505.0                    1.462703e+13   

        Avg_EV_4_Quarters_Rolling   EBITDAEV  
268                  2.007452e+11  10.056136  
269                  2.006998e+11  10.059037  
270                  2.006525e+11  10.062035  
271                  2.006129e+11  10.064652  
272                  2.005663e+1

C:\Users\asas9\AppData\Local\Temp\ipykernel_28296\366518415.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()


In [47]:
export_columns = ['Date', 'ticker', 'EBITDAEV']
# 导出到 CSV 文件
df_filtered[export_columns].to_csv('EBITDAEV.csv', index=False)

自由現金流對市值比

In [49]:
import pandas as pd

# Load the uploaded CSV file
file_path = '50合併.csv'
df = pd.read_csv(file_path)

# Convert Date column to datetime format and sort by date and ticker
df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')
df = df.sort_values(by=['ticker', 'Date'])

# Convert relevant columns to numeric types to avoid type errors
df['OANCF'] = pd.to_numeric(df['OANCF'], errors='coerce')
df['CAPXQ'] = pd.to_numeric(df['CAPXQ'], errors='coerce')
df['DVQ'] = pd.to_numeric(df['DVQ'], errors='coerce')
df['CSHOQ'] = pd.to_numeric(df['CSHOQ'], errors='coerce')
df['Adj_Close'] = pd.to_numeric(df['Adj_Close'], errors='coerce')

# Ensure each group has a continuous business day index
df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()

# Calculate rolling sum of (OANCF - CAPXQ - DVQ) over the past 4 quarters (approx. 252 trading days)
df['Total_Free_Cash_Flow_4_Quarters_Rolling'] = df.groupby('ticker').apply(
    lambda x: (x['OANCF'] - x['CAPXQ'] - x['DVQ']).rolling(window=252, min_periods=252).sum()
).reset_index(level=0, drop=True)

# Calculate rolling average of CSHOQ over the past 4 quarters (approx. 252 trading days)
df['Avg_CSHOQ_4_Quarters_Rolling'] = df.groupby('ticker')['CSHOQ'].rolling(window=252, min_periods=252).mean().reset_index(level=0, drop=True)

# Calculate Free Cash Flow to Price ratio (FCFP) based on the provided formula
df['FCFP'] = df['Total_Free_Cash_Flow_4_Quarters_Rolling'] / (df['Adj_Close'] * df['Avg_CSHOQ_4_Quarters_Rolling'])

# Filter the DataFrame to remove rows with missing values in the calculated columns
df_filtered = df[['Date', 'ticker', 'Total_Free_Cash_Flow_4_Quarters_Rolling', 'Avg_CSHOQ_4_Quarters_Rolling', 'Adj_Close', 'FCFP']].dropna()

# Display the filtered results
print(df_filtered)

C:\Users\asas9\AppData\Local\Temp\ipykernel_28296\2000640030.py:5: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


             Date  ticker  Total_Free_Cash_Flow_4_Quarters_Rolling  \
268    2017-10-20  1101.0                             7.660719e+11   
269    2017-10-23  1101.0                             7.650780e+11   
270    2017-10-24  1101.0                             7.640840e+11   
271    2017-10-25  1101.0                             7.630901e+11   
272    2017-10-26  1101.0                             7.620961e+11   
...           ...     ...                                      ...   
100266 2024-10-04  6505.0                             1.297907e+12   
100267 2024-10-07  6505.0                             1.290404e+12   
100268 2024-10-08  6505.0                             1.282901e+12   
100269 2024-10-09  6505.0                             1.275399e+12   
100271 2024-10-11  6505.0                             1.267896e+12   

        Avg_CSHOQ_4_Quarters_Rolling  Adj_Close       FCFP  
268                     3.692176e+09      18.41  11.270246  
269                     3.692176e+09 

C:\Users\asas9\AppData\Local\Temp\ipykernel_28296\2000640030.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()
C:\Users\asas9\AppData\Local\Temp\ipykernel_28296\2000640030.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['Total_Free_Cash_Flow_4_Quarters_Rolling'] = df.groupby('ticker').apply(

In [51]:
export_columns = ['Date', 'ticker', 'FCFP']
# 导出到 CSV 文件
df_filtered[export_columns].to_csv('FCFP.csv', index=False)

 市淨率

In [53]:
import pandas as pd

# Load the uploaded CSV file
file_path = '50合併.csv'
df = pd.read_csv(file_path)

# Convert Date column to datetime format and sort by date and ticker
df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')
df = df.sort_values(by=['ticker', 'Date'])

# Convert relevant columns to numeric types to avoid type errors
df['CEQQ'] = pd.to_numeric(df['CEQQ'], errors='coerce')
df['CSHOQ'] = pd.to_numeric(df['CSHOQ'], errors='coerce')
df['Adj_Close'] = pd.to_numeric(df['Adj_Close'], errors='coerce')

# Ensure each group has a continuous business day index
df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()

# Calculate Book-to-Price ratio (BP) based on the provided formula
df['BP'] = df['CEQQ'] / (df['CSHOQ'] * df['Adj_Close'])

# Filter the DataFrame to remove rows with missing values in the calculated columns
df_filtered = df[['Date', 'ticker', 'CEQQ', 'CSHOQ', 'Adj_Close', 'BP']].dropna()

# Display the filtered results
print(df_filtered)

C:\Users\asas9\AppData\Local\Temp\ipykernel_28296\1715699233.py:5: DtypeWarning: Columns (2,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


             Date  ticker          CEQQ         CSHOQ  Adj_Close        BP
0      2016-10-11  1101.0  1.467727e+11  3.692176e+09      18.79  2.115613
1      2016-10-12  1101.0  1.467727e+11  3.692176e+09      18.79  2.115613
2      2016-10-13  1101.0  1.467727e+11  3.692176e+09      18.24  2.179406
3      2016-10-14  1101.0  1.467727e+11  3.692176e+09      18.68  2.128071
4      2016-10-17  1101.0  1.467727e+11  3.692176e+09      18.79  2.115613
...           ...     ...           ...           ...        ...       ...
100266 2024-10-04  6505.0  3.321784e+11  9.525960e+09      54.50  0.639832
100267 2024-10-07  6505.0  3.321784e+11  9.525960e+09      55.00  0.634016
100268 2024-10-08  6505.0  3.321784e+11  9.525960e+09      54.20  0.643374
100269 2024-10-09  6505.0  3.321784e+11  9.525960e+09      52.30  0.666747
100271 2024-10-11  6505.0  3.321784e+11  9.525960e+09      52.00  0.670594

[93514 rows x 6 columns]


C:\Users\asas9\AppData\Local\Temp\ipykernel_28296\1715699233.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()


In [55]:
export_columns = ['Date', 'ticker', 'BP']
# 导出到 CSV 文件
df_filtered[export_columns].to_csv('BP.csv', index=False)

市值的對數

In [57]:
import pandas as pd
import numpy as np

# Load the uploaded CSV file
file_path = '50合併.csv'
df = pd.read_csv(file_path)

# Convert Date column to datetime format and sort by date and ticker
df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')
df = df.sort_values(by=['ticker', 'Date'])

# Convert relevant columns to numeric types to avoid type errors
df['CSHOQ'] = pd.to_numeric(df['CSHOQ'], errors='coerce')
df['Adj_Close'] = pd.to_numeric(df['Adj_Close'], errors='coerce')

# Ensure each group has a continuous business day index
df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()

# Calculate Market Capitalization (MCap) and its logarithm based on the provided formula
# Calculate market capitalization
df['MCap'] = df['CSHOQ'] * df['Adj_Close']

# Calculate the natural logarithm of the market capitalization
df['log_MCap'] = np.log(df['MCap'])

# Filter the DataFrame to remove rows with missing values in the calculated columns
df_filtered_mcap = df[['Date', 'ticker', 'CSHOQ', 'Adj_Close', 'MCap', 'log_MCap']].dropna()

# Display the filtered results
print(df_filtered_mcap)

C:\Users\asas9\AppData\Local\Temp\ipykernel_28296\2239441258.py:6: DtypeWarning: Columns (2,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


             Date  ticker         CSHOQ  Adj_Close          MCap   log_MCap
0      2016-10-11  1101.0  3.692176e+09      18.79  6.937599e+10  24.962807
1      2016-10-12  1101.0  3.692176e+09      18.79  6.937599e+10  24.962807
2      2016-10-13  1101.0  3.692176e+09      18.24  6.734529e+10  24.933099
3      2016-10-14  1101.0  3.692176e+09      18.68  6.896985e+10  24.956935
4      2016-10-17  1101.0  3.692176e+09      18.79  6.937599e+10  24.962807
...           ...     ...           ...        ...           ...        ...
100266 2024-10-04  6505.0  9.525960e+09      54.50  5.191648e+11  26.975487
100267 2024-10-07  6505.0  9.525960e+09      55.00  5.239278e+11  26.984620
100268 2024-10-08  6505.0  9.525960e+09      54.20  5.163070e+11  26.969967
100269 2024-10-09  6505.0  9.525960e+09      52.30  4.982077e+11  26.934283
100271 2024-10-11  6505.0  9.525960e+09      52.00  4.953499e+11  26.928530

[93514 rows x 6 columns]


C:\Users\asas9\AppData\Local\Temp\ipykernel_28296\2239441258.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()


In [61]:
export_columns = ['Date', 'ticker', 'log_MCap']
# 导出到 CSV 文件
df_filtered_mcap[export_columns].to_csv('log_MCap.csv', index=False)

銷售額的對數

In [63]:
import pandas as pd
import numpy as np

# Load the uploaded CSV file
file_path = '50合併.csv'
df = pd.read_csv(file_path)

# Convert Date column to datetime format and sort by date and ticker
df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')
df = df.sort_values(by=['ticker', 'Date'])

# Convert relevant columns to numeric types to avoid type errors
df['SALEQ'] = pd.to_numeric(df['SALEQ'], errors='coerce')

# Ensure each group has a continuous business day index
df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()

# Calculate rolling sum of SALESQ over the past 4 quarters (approx. 252 trading days)
df['TTM_Sales'] = df.groupby('ticker')['SALEQ'].rolling(window=252, min_periods=252).sum().reset_index(level=0, drop=True)

# Calculate the natural logarithm of the TTM Sales
df['log_TTMSales'] = np.log(df['TTM_Sales'])

# Filter the DataFrame to remove rows with missing values in the calculated columns for TTM Sales
df_filtered_ttmsales = df[['Date', 'ticker', 'TTM_Sales', 'log_TTMSales']].dropna()

# Display the filtered results for TTM Sales
print(df_filtered_ttmsales)

C:\Users\asas9\AppData\Local\Temp\ipykernel_28296\1375968686.py:6: DtypeWarning: Columns (2,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


             Date  ticker     TTM_Sales  log_TTMSales
268    2017-10-20  1101.0  2.018721e+12     28.333485
269    2017-10-23  1101.0  2.018847e+12     28.333548
270    2017-10-24  1101.0  2.018973e+12     28.333610
271    2017-10-25  1101.0  2.019099e+12     28.333672
272    2017-10-26  1101.0  2.019225e+12     28.333735
...           ...     ...           ...           ...
100266 2024-10-04  6505.0  1.469524e+13     30.318545
100267 2024-10-07  6505.0  1.467818e+13     30.317383
100268 2024-10-08  6505.0  1.466113e+13     30.316221
100269 2024-10-09  6505.0  1.464408e+13     30.315057
100271 2024-10-11  6505.0  1.462703e+13     30.313892

[81504 rows x 4 columns]


C:\Users\asas9\AppData\Local\Temp\ipykernel_28296\1375968686.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()


In [65]:
export_columns = ['Date', 'ticker', 'log_TTMSales']
# 导出到 CSV 文件
df_filtered_ttmsales[export_columns].to_csv('log_TTMSales.csv', index=False)

5日價格反轉

In [69]:
import pandas as pd

# Load the uploaded CSV file
file_path = '50合併.csv'
df = pd.read_csv(file_path)

# Convert Date column to datetime format and sort by date and ticker
df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')
df = df.sort_values(by=['ticker', 'Date'])

# Convert relevant columns to numeric types to avoid type errors
df['Adj_Close'] = pd.to_numeric(df['Adj_Close'], errors='coerce')

# Ensure each group has a continuous business day index
df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()

# Calculate 5-day Price Reversal (PM5D)
df['PM5D'] = (df.groupby('ticker')['Adj_Close'].shift(5) - df['Adj_Close']) / df.groupby('ticker')['Adj_Close'].shift(5)

# Filter the DataFrame to remove rows with missing values in the calculated columns for PM5D
df_filtered_pm5d = df[['Date', 'ticker', 'Adj_Close', 'PM5D']].dropna()

# Display the filtered results for PM5D
print(df_filtered_pm5d)

C:\Users\asas9\AppData\Local\Temp\ipykernel_28296\2403878929.py:5: DtypeWarning: Columns (2,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


             Date  ticker  Adj_Close      PM5D
5      2016-10-18  1101.0      18.97 -0.009580
6      2016-10-19  1101.0      19.46 -0.035657
7      2016-10-20  1101.0      19.23 -0.054276
8      2016-10-21  1101.0      19.13 -0.024090
9      2016-10-24  1101.0      19.28 -0.026078
...           ...     ...        ...       ...
100266 2024-10-04  6505.0      54.50 -0.064453
100267 2024-10-07  6505.0      55.00 -0.084813
100268 2024-10-08  6505.0      54.20 -0.022642
100269 2024-10-09  6505.0      52.30  0.007590
100271 2024-10-11  6505.0      52.00  0.005736

[93249 rows x 4 columns]


C:\Users\asas9\AppData\Local\Temp\ipykernel_28296\2403878929.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()


In [71]:
export_columns = ['Date', 'ticker', 'PM5D']
# 导出到 CSV 文件
df_filtered_pm5d[export_columns].to_csv('PM5D.csv', index=False)

In [ ]:
 9個月價格動能

In [73]:
import pandas as pd

# Load the uploaded CSV file
file_path = '50合併.csv'
df = pd.read_csv(file_path)

# Convert Date column to datetime format and sort by date and ticker
df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')
df = df.sort_values(by=['ticker', 'Date'])

# Convert relevant columns to numeric types to avoid type errors
df['Adj_Close'] = pd.to_numeric(df['Adj_Close'], errors='coerce')

# Ensure each group has a continuous business day index
df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()

# Calculate 9-month Price Momentum (PM9M) using the adjusted close prices
# Shift by approximately 9 months (assuming 21 trading days per month, so 189 trading days)
df['PM9M'] = (df['Adj_Close'] - df.groupby('ticker')['Adj_Close'].shift(189)) / df.groupby('ticker')['Adj_Close'].shift(189)

# Filter the DataFrame to remove rows with missing values in the calculated columns for PM9M
df_filtered_pm9m = df[['Date', 'ticker', 'Adj_Close', 'PM9M']].dropna()

# Display the filtered results for PM9M
print(df_filtered_pm9m)

C:\Users\asas9\AppData\Local\Temp\ipykernel_28296\287458747.py:5: DtypeWarning: Columns (2,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


             Date  ticker  Adj_Close      PM9M
203    2017-07-21  1101.0      18.81  0.001064
204    2017-07-24  1101.0      18.76 -0.001597
205    2017-07-25  1101.0      18.76  0.028509
206    2017-07-26  1101.0      18.73  0.002677
207    2017-07-27  1101.0      19.11  0.017030
...           ...     ...        ...       ...
100266 2024-10-04  6505.0      54.50 -0.321971
100267 2024-10-07  6505.0      55.00 -0.307392
100268 2024-10-08  6505.0      54.20 -0.302984
100269 2024-10-09  6505.0      52.30 -0.331544
100271 2024-10-11  6505.0      52.00 -0.344345

[84413 rows x 4 columns]


C:\Users\asas9\AppData\Local\Temp\ipykernel_28296\287458747.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()


In [75]:
export_columns = ['Date', 'ticker', 'PM9M']
# 导出到 CSV 文件
df_filtered_pm9m[export_columns].to_csv('PM9M.csv', index=False)

1個月高低波動率比值 

In [111]:
import pandas as pd
import numpy as np

# Load the uploaded CSV file
file_path = '50合併.csv'
df = pd.read_csv(file_path)

# Convert Date column to datetime format and sort by date and ticker
df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')
df = df.sort_values(by=['ticker', 'Date'])

# Convert relevant columns to numeric types to avoid type errors
df['Adj_Close'] = pd.to_numeric(df['Adj_Close'], errors='coerce')

# Ensure each group has a continuous business day index
df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()

# Calculate rolling 1-month high and low (approx. 21 trading days), excluding the current day for each ticker
df['HIGHM'] = df.groupby('ticker', group_keys=False)['Adj_Close'].apply(lambda x: x.shift(1).rolling(window=21, min_periods=21).max())
df['LOWM'] = df.groupby('ticker', group_keys=False)['Adj_Close'].apply(lambda x: x.shift(1).rolling(window=21, min_periods=21).min())

# Drop rows where HIGHM or LOWM are NaN due to insufficient past data
df = df.dropna(subset=['HIGHM', 'LOWM']).reset_index(drop=True)

# Calculate HL1M based on rolling high and low values, ensuring calculation only for valid rows
df['HL1M'] = (df['HIGHM'] - df['Adj_Close']) / (df['Adj_Close'] - df['LOWM'])

# Filter the DataFrame to remove rows with missing values in the calculated columns for HL1M
df_filtered_hl1m = df[['Date', 'ticker', 'Adj_Close', 'HIGHM', 'LOWM', 'HL1M']].dropna()

# Display the filtered results for HL1M
df_filtered_hl1m = df_filtered_hl1m.sort_values(by=['ticker', 'Date'])
print(df_filtered_hl1m)

C:\Users\asas9\AppData\Local\Temp\ipykernel_28296\1740645994.py:6: DtypeWarning: Columns (2,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


            Date  ticker  Adj_Close  HIGHM   LOWM      HL1M
0     2016-11-09  1101.0      18.92  19.73  18.24  1.191176
1     2016-11-10  1101.0      19.88  19.73  18.24 -0.091463
2     2016-11-11  1101.0      19.54  19.88  18.24  0.261538
3     2016-11-14  1101.0      19.31  19.88  18.68  0.904762
4     2016-11-15  1101.0      19.13  19.88  18.71  1.785714
...          ...     ...        ...    ...    ...       ...
92401 2024-10-04  6505.0      54.50  55.10  47.40  0.084507
92402 2024-10-07  6505.0      55.00  54.80  47.40 -0.026316
92403 2024-10-08  6505.0      54.20  55.00  47.40  0.117647
92404 2024-10-09  6505.0      52.30  55.00  47.40  0.551020
92405 2024-10-11  6505.0      52.00  55.00  47.40  0.652174

[92401 rows x 6 columns]


C:\Users\asas9\AppData\Local\Temp\ipykernel_28296\1740645994.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()


In [113]:
export_columns = ['Date', 'ticker', 'HL1M']
# 导出到 CSV 文件
df_filtered_hl1m[export_columns].to_csv('HL1M.csv', index=False)

1個月價格反轉

In [121]:
import pandas as pd
import numpy as np

# Load the uploaded CSV file
file_path = '50合併.csv'
df = pd.read_csv(file_path)

# Convert Date column to datetime format and sort by date and ticker
df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')
df = df.sort_values(by=['ticker', 'Date'])

# Convert relevant columns to numeric types to avoid type errors
df['Adj_Close'] = pd.to_numeric(df['Adj_Close'], errors='coerce')

# Ensure each group has a continuous business day index, preserving individual date ranges for each ticker
df = df.groupby('ticker').apply(lambda x: x.set_index('Date').asfreq('B').reset_index()).reset_index(drop=True)

# Calculate adjusted close prices for the current month (days 22 to 42) and previous month (days 1 to 21)
df['Previous_Month_Close'] = df.groupby('ticker', group_keys=False)['Adj_Close'].apply(lambda x: x.shift(21).rolling(window=21, min_periods=21).mean())
df['Current_Month_Close'] = df.groupby('ticker', group_keys=False)['Adj_Close'].apply(lambda x: x.shift(1).rolling(window=21, min_periods=21).mean())

# Calculate 1-month Price Reversal (PM1M) on a daily rolling basis starting from day 42
df['PM1M'] = (df['Current_Month_Close'] - df['Previous_Month_Close']) / df['Previous_Month_Close']

# Filter the DataFrame to remove rows with missing values in the calculated columns for PM1M
df_filtered_pm1m = df[['Date', 'ticker', 'Adj_Close', 'Current_Month_Close', 'Previous_Month_Close', 'PM1M']].dropna()

# Display the filtered results for PM1M
df_filtered_pm1m = df_filtered_pm1m.sort_values(by=['ticker', 'Date'])
print(df_filtered_pm1m)

C:\Users\asas9\AppData\Local\Temp\ipykernel_28296\2455823385.py:6: DtypeWarning: Columns (2,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


             Date  ticker  Adj_Close  Current_Month_Close  \
41     2016-12-07  1101.0      19.31            19.223333   
42     2016-12-08  1101.0      19.80            19.212381   
43     2016-12-09  1101.0      19.78            19.254286   
44     2016-12-12  1101.0      19.75            19.249524   
45     2016-12-13  1101.0      19.73            19.259524   
...           ...     ...        ...                  ...   
100266 2024-10-04  6505.0      54.50            51.102381   
100267 2024-10-07  6505.0      55.00            51.073810   
100268 2024-10-08  6505.0      54.20            51.083333   
100269 2024-10-09  6505.0      52.30            51.207143   
100271 2024-10-11  6505.0      52.00            51.302381   

        Previous_Month_Close      PM1M  
41                 19.150952  0.003779  
42                 19.157143  0.002883  
43                 19.209048  0.002355  
44                 19.270952 -0.001112  
45                 19.300952 -0.002146  
...                  

C:\Users\asas9\AppData\Local\Temp\ipykernel_28296\2455823385.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('ticker').apply(lambda x: x.set_index('Date').asfreq('B').reset_index()).reset_index(drop=True)


In [123]:
export_columns = ['Date', 'ticker', 'PM1M']
# 导出到 CSV 文件
df_filtered_pm1m[export_columns].to_csv('PM1M.csv', index=False)

12個月減去1個月的動能

In [125]:
import pandas as pd
import numpy as np

# Load the uploaded CSV file
file_path = '50合併.csv'
df = pd.read_csv(file_path)

# Convert Date column to datetime format and sort by date and ticker
df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')
df = df.sort_values(by=['ticker', 'Date'])

# Convert relevant columns to numeric types to avoid type errors
df['Adj_Close'] = pd.to_numeric(df['Adj_Close'], errors='coerce')

# Ensure each group has a continuous business day index, preserving individual date ranges for each ticker
df = df.groupby('ticker').apply(lambda x: x.set_index('Date').asfreq('B').reset_index()).reset_index(drop=True)

# Calculate adjusted close prices for the previous month (shifted by 1 month) and 12 months ago
# Previous month (shifted by 21 trading days)
df['Previous_Month_Close'] = df.groupby('ticker', group_keys=False)['Adj_Close'].apply(lambda x: x.shift(21))
# 12 months ago (shifted by 252 trading days, approx. 12 months)
df['Twelve_Month_Close'] = df.groupby('ticker', group_keys=False)['Adj_Close'].apply(lambda x: x.shift(252))

# Calculate 12M - 1M Price Momentum (PM12M1M) on a daily rolling basis
df['PM12M1M'] = (df['Previous_Month_Close'] - df['Twelve_Month_Close']) / df['Twelve_Month_Close']

# Filter the DataFrame to remove rows with missing values in the calculated columns for PM12M1M
df_filtered_pm12m1m = df[['Date', 'ticker', 'Adj_Close', 'Previous_Month_Close', 'Twelve_Month_Close', 'PM12M1M']].dropna()

# Display the filtered results for PM12M1M
df_filtered_pm12m1m = df_filtered_pm12m1m.sort_values(by=['ticker', 'Date'])
print(df_filtered_pm12m1m)

C:\Users\asas9\AppData\Local\Temp\ipykernel_28296\4069550168.py:6: DtypeWarning: Columns (2,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


             Date  ticker  Adj_Close  Previous_Month_Close  \
269    2017-10-23  1101.0      18.35                 18.44   
270    2017-10-24  1101.0      18.22                 18.41   
271    2017-10-25  1101.0      18.19                 18.35   
272    2017-10-26  1101.0      18.16                 18.41   
273    2017-10-27  1101.0      18.16                 18.49   
...           ...     ...        ...                   ...   
100266 2024-10-04  6505.0      54.50                 55.10   
100267 2024-10-07  6505.0      55.00                 54.80   
100268 2024-10-08  6505.0      54.20                 51.60   
100269 2024-10-09  6505.0      52.30                 50.30   
100271 2024-10-11  6505.0      52.00                 50.90   

        Twelve_Month_Close   PM12M1M  
269                  18.79 -0.018627  
270                  18.79 -0.020224  
271                  18.24  0.006031  
272                  18.68 -0.014454  
273                  18.79 -0.015966  
...                  

C:\Users\asas9\AppData\Local\Temp\ipykernel_28296\4069550168.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('ticker').apply(lambda x: x.set_index('Date').asfreq('B').reset_index()).reset_index(drop=True)


In [127]:
export_columns = ['Date', 'ticker', 'PM12M1M']
# 导出到 CSV 文件
df_filtered_pm12m1m[export_columns].to_csv('PM12M1M.csv', index=False)

**可持續增長率

In [28]:
import pandas as pd
import numpy as np

# Load the uploaded CSV file
file_path = '50合併.csv'
df = pd.read_csv(file_path)

# Convert Date column to datetime format and sort by date and ticker
df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')
df = df.sort_values(by=['ticker', 'Date'])

# Convert relevant columns to numeric types to avoid type errors
df['EPS'] = pd.to_numeric(df['EPS'], errors='coerce')
df['DVQ'] = pd.to_numeric(df['DVQ'], errors='coerce')
df['CEQQ'] = pd.to_numeric(df['CEQQ'], errors='coerce')
df['CSHOQ'] = pd.to_numeric(df['CSHOQ'], errors='coerce')

# Ensure each group has a continuous business day index, preserving individual date ranges for each ticker
df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()

# Calculate Retention Ratio based on the past 4 quarters (approx. 252 trading days)
df['TTM_EPS'] = df.groupby('ticker', group_keys=False)['EPS'].apply(lambda x: x.shift(1).rolling(window=252, min_periods=252).sum())
df['TTM_DVQ'] = df.groupby('ticker', group_keys=False)['DVQ'].apply(lambda x: x.shift(1).rolling(window=252, min_periods=252).sum())
df['RetentionRatio'] = 1 - (df['TTM_DVQ'] / df['TTM_EPS'])

# Calculate TTMCEPS (Trailing Twelve Month Common Equity per Share) based on the past 4 quarters
df['TTM_CEQQ'] = df.groupby('ticker', group_keys=False)['CEQQ'].apply(lambda x: x.shift(1).rolling(window=252, min_periods=252).sum())
df['TTM_CSHOQ'] = df.groupby('ticker', group_keys=False)['CSHOQ'].apply(lambda x: x.shift(1).rolling(window=252, min_periods=252).sum())
df['TTMCEPS'] = df['TTM_CEQQ'] / (df['TTM_CSHOQ'] / 4)

# Calculate Sustainable Growth Rate (SusGrwRate)
df['SusGrwRate'] = df['RetentionRatio'] * df['TTMCEPS']

# Filter the DataFrame to remove rows with missing values in the calculated columns
df_filtered_susgrwrate = df[['Date', 'ticker', 'EPS', 'DVQ', 'CEQQ', 'CSHOQ', 'TTM_EPS', 'TTM_DVQ', 'RetentionRatio', 'TTMCEPS', 'SusGrwRate']].dropna()

# Display the filtered results for Sustainable Growth Rate
df_filtered_susgrwrate = df_filtered_susgrwrate.sort_values(by=['ticker', 'Date'])
print(df_filtered_susgrwrate)


C:\Users\asas9\AppData\Local\Temp\ipykernel_7732\2348592915.py:6: DtypeWarning: Columns (2,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


             Date  ticker   EPS  DVQ          CEQQ         CSHOQ  TTM_EPS  \
269    2017-10-23  1101.0  0.58  0.0  1.450000e+11  3.692176e+09   125.13   
270    2017-10-24  1101.0  0.58  0.0  1.450000e+11  3.692176e+09   125.17   
271    2017-10-25  1101.0  0.58  0.0  1.450000e+11  3.692176e+09   125.21   
272    2017-10-26  1101.0  0.58  0.0  1.450000e+11  3.692176e+09   125.25   
273    2017-10-27  1101.0  0.58  0.0  1.450000e+11  3.692176e+09   125.29   
...           ...     ...   ...  ...           ...           ...      ...   
100266 2024-10-04  6505.0  0.29  0.0  3.320000e+11  9.525960e+09   204.63   
100267 2024-10-07  6505.0  0.29  0.0  3.320000e+11  9.525960e+09   205.04   
100268 2024-10-08  6505.0  0.29  0.0  3.320000e+11  9.525960e+09   205.45   
100269 2024-10-09  6505.0  0.29  0.0  3.320000e+11  9.525960e+09   205.86   
100271 2024-10-11  6505.0  0.29  0.0  3.320000e+11  9.525960e+09   206.27   

        TTM_DVQ  RetentionRatio     TTMCEPS  SusGrwRate  
269         0.0  

C:\Users\asas9\AppData\Local\Temp\ipykernel_7732\2348592915.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()


In [30]:
export_columns = ['Date', 'ticker', 'SusGrwRate']
# 导出到 CSV 文件
df_filtered_susgrwrate[export_columns].to_csv('SusGrwRate.csv', index=False)

EPS 成長率

In [5]:
import pandas as pd
import numpy as np

# Load the uploaded CSV file
file_path = '50合併.csv'
df = pd.read_csv(file_path)

# Convert Date column to datetime format and sort by date and ticker
df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')
df = df.sort_values(by=['ticker', 'Date'])

# Convert relevant columns to numeric types to avoid type errors
df['Adj_Close'] = pd.to_numeric(df['Adj_Close'], errors='coerce')
df['EPS'] = pd.to_numeric(df['EPS'], errors='coerce')

# Ensure each group has a continuous business day index, preserving individual date ranges for each ticker
df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()

# Calculate TTMEPS (Trailing Twelve Months Earnings Per Share) based on the past 4 quarters (approx. 252 trading days)
df['TTMEPS'] = df.groupby('ticker', group_keys=False)['EPS'].apply(lambda x: x.shift(1).rolling(window=252, min_periods=252).sum())

# Calculate TTMEPS from one year ago (approx. 252 trading days)
df['TTMEPS_1Y_ago'] = df.groupby('ticker', group_keys=False)['TTMEPS'].shift(252)

# Calculate Change in 1-Year EPS (Chg1YEPS)
df['Chg1YEPS'] = (df['TTMEPS'] - df['TTMEPS_1Y_ago']) / df['TTMEPS_1Y_ago']

# Filter the DataFrame to remove rows with missing values in the calculated columns
df_filtered_chg1yeps = df[['Date', 'ticker', 'Adj_Close', 'TTMEPS', 'TTMEPS_1Y_ago', 'Chg1YEPS']].dropna()

# Display the filtered results for Change in 1-Year EPS
df_filtered_chg1yeps = df_filtered_chg1yeps.sort_values(by=['ticker', 'Date'])
print(df_filtered_chg1yeps)


C:\Users\asas9\AppData\Local\Temp\ipykernel_18500\46015242.py:6: DtypeWarning: Columns (2,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


             Date  ticker  Adj_Close  TTMEPS  TTMEPS_1Y_ago  Chg1YEPS
536    2018-10-31  1101.0      21.44  202.38         125.13  0.617358
537    2018-11-01  1101.0      22.03  203.31         125.17  0.624271
538    2018-11-02  1101.0      22.52  204.24         125.21  0.631180
539    2018-11-05  1101.0      21.84  205.17         125.25  0.638084
540    2018-11-06  1101.0      21.69  206.10         125.29  0.644984
...           ...     ...        ...     ...            ...       ...
100266 2024-10-04  6505.0      54.50  204.63          27.08  6.556499
100267 2024-10-07  6505.0      55.00  205.04          25.18  7.142971
100268 2024-10-08  6505.0      54.20  205.45          23.28  7.825172
100269 2024-10-09  6505.0      52.30  205.86          21.38  8.628625
100271 2024-10-11  6505.0      52.00  206.27          19.48  9.588809

[67978 rows x 6 columns]


C:\Users\asas9\AppData\Local\Temp\ipykernel_18500\46015242.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()


In [7]:
export_columns = ['Date', 'ticker', 'Chg1YEPS']
# 导出到 CSV 文件
df_filtered_chg1yeps[export_columns].to_csv('Chg1YEPS.csv', index=False)

資產周轉率變動

In [9]:
import pandas as pd
import numpy as np

# Load the uploaded CSV file
file_path = '50合併.csv'
df = pd.read_csv(file_path)

# Convert Date column to datetime format and sort by date and ticker
df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')
df = df.sort_values(by=['ticker', 'Date'])

# Convert relevant columns to numeric types to avoid type errors
df['SALEQ'] = pd.to_numeric(df['SALEQ'], errors='coerce')
df['ATQ'] = pd.to_numeric(df['ATQ'], errors='coerce')

# Ensure each group has a continuous business day index, preserving individual date ranges for each ticker
df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()

# Calculate TTMSALES (Trailing Twelve Months Sales) based on the past 4 quarters (approx. 252 trading days)
df['TTMSALES'] = df.groupby('ticker', group_keys=False)['SALEQ'].apply(lambda x: x.shift(1).rolling(window=252, min_periods=252).sum())

# Calculate AvgAst (Average Total Assets) based on the past 4 quarters (approx. 252 trading days)
df['AvgAst'] = df.groupby('ticker', group_keys=False)['ATQ'].apply(lambda x: x.shift(1).rolling(window=252, min_periods=252).mean())

# Calculate TTMSALES and AvgAst from one year ago (approx. 252 trading days)
df['TTMSALES_1Y_ago'] = df.groupby('ticker', group_keys=False)['TTMSALES'].shift(252)
df['AvgAst_1Y_ago'] = df.groupby('ticker', group_keys=False)['AvgAst'].shift(252)

# Calculate Change in 1-Year Asset Turnover (Chg1YAstTo)
df['Chg1YAstTo'] = (df['TTMSALES'] / df['AvgAst']) - (df['TTMSALES_1Y_ago'] / df['AvgAst_1Y_ago'])

# Filter the DataFrame to remove rows with missing values in the calculated columns
df_filtered_chg1yastto = df[['Date', 'ticker', 'SALEQ', 'ATQ', 'TTMSALES', 'AvgAst', 'TTMSALES_1Y_ago', 'AvgAst_1Y_ago', 'Chg1YAstTo']].dropna()

# Display the filtered results for Change in 1-Year Asset Turnover
df_filtered_chg1yastto = df_filtered_chg1yastto.sort_values(by=['ticker', 'Date'])
print(df_filtered_chg1yastto)


C:\Users\asas9\AppData\Local\Temp\ipykernel_18500\115106120.py:6: DtypeWarning: Columns (2,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


             Date  ticker         SALEQ           ATQ      TTMSALES  \
536    2018-10-31  1101.0  1.175330e+10  3.099022e+11  2.579530e+12   
537    2018-11-01  1101.0  1.175330e+10  3.099022e+11  2.582896e+12   
538    2018-11-02  1101.0  1.175330e+10  3.099022e+11  2.586261e+12   
539    2018-11-05  1101.0  1.175330e+10  3.099022e+11  2.589627e+12   
540    2018-11-06  1101.0  1.175330e+10  3.099022e+11  2.592993e+12   
...           ...     ...           ...           ...           ...   
100266 2024-10-04  6505.0  5.332454e+10  4.286200e+11  1.471229e+13   
100267 2024-10-07  6505.0  5.332454e+10  4.286200e+11  1.469524e+13   
100268 2024-10-08  6505.0  5.332454e+10  4.286200e+11  1.467818e+13   
100269 2024-10-09  6505.0  5.332454e+10  4.286200e+11  1.466113e+13   
100271 2024-10-11  6505.0  5.332454e+10  4.286200e+11  1.464408e+13   

              AvgAst  TTMSALES_1Y_ago  AvgAst_1Y_ago  Chg1YAstTo  
536     2.814545e+11     2.018721e+12   2.637138e+11    1.510030  
537     2.816

C:\Users\asas9\AppData\Local\Temp\ipykernel_18500\115106120.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()


In [11]:
export_columns = ['Date', 'ticker', 'Chg1YAstTo']
# 导出到 CSV 文件
df_filtered_chg1yastto[export_columns].to_csv('Chg1YAstTo.csv', index=False)

淨利潤率

In [2]:
import pandas as pd
import numpy as np

# Load the uploaded CSV file
file_path = '50合併.csv'
df = pd.read_csv(file_path)

# Convert Date column to datetime format and sort by date and ticker
df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')
df = df.sort_values(by=['ticker', 'Date'])

# Convert relevant columns to numeric types to avoid type errors
df['SALEQ'] = pd.to_numeric(df['SALEQ'], errors='coerce')
df['IBQ'] = pd.to_numeric(df['IBQ'], errors='coerce')

# Ensure each group has a continuous business day index, preserving individual date ranges for each ticker
df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()

# Calculate TTM Sales (Trailing Twelve Months Sales) based on the past 4 quarters (approx. 252 trading days)
df['TTMSales'] = df.groupby('ticker', group_keys=False)['SALEQ'].apply(lambda x: x.shift(1).rolling(window=252, min_periods=252).sum())

# Calculate TTM Income Before Extraordinary Items (TTMIB) based on the past 4 quarters (approx. 252 trading days)
df['TTMIB'] = df.groupby('ticker', group_keys=False)['IBQ'].apply(lambda x: x.shift(1).rolling(window=252, min_periods=252).sum())

# Calculate Net Profit Margin (NetProfitMargin)
df['NetProfitMargin'] = df['TTMIB'] / df['TTMSales']

# Filter the DataFrame to remove rows with missing values in the calculated columns
df_filtered_npm = df[['Date', 'ticker', 'SALEQ', 'IBQ', 'TTMSales', 'TTMIB', 'NetProfitMargin']].dropna()

# Display the filtered results for Net Profit Margin
df_filtered_npm = df_filtered_npm.sort_values(by=['ticker', 'Date'])
print(df_filtered_npm)


C:\Users\asas9\AppData\Local\Temp\ipykernel_7732\3383108178.py:6: DtypeWarning: Columns (2,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


             Date  ticker         SALEQ           IBQ      TTMSales  \
269    2017-10-23  1101.0  8.387381e+09  4.079056e+09  2.018721e+12   
270    2017-10-24  1101.0  8.387381e+09  4.079056e+09  2.018847e+12   
271    2017-10-25  1101.0  8.387381e+09  4.079056e+09  2.018973e+12   
272    2017-10-26  1101.0  8.387381e+09  4.079056e+09  2.019099e+12   
273    2017-10-27  1101.0  8.387381e+09  4.079056e+09  2.019225e+12   
...           ...     ...           ...           ...           ...   
100266 2024-10-04  6505.0  5.332454e+10  3.260798e+09  1.471229e+13   
100267 2024-10-07  6505.0  5.332454e+10  3.260798e+09  1.469524e+13   
100268 2024-10-08  6505.0  5.332454e+10  3.260798e+09  1.467818e+13   
100269 2024-10-09  6505.0  5.332454e+10  3.260798e+09  1.466113e+13   
100271 2024-10-11  6505.0  5.332454e+10  3.260798e+09  1.464408e+13   

               TTMIB  NetProfitMargin  
269     8.475890e+11         0.419864  
270     8.481998e+11         0.420141  
271     8.488107e+11       

C:\Users\asas9\AppData\Local\Temp\ipykernel_7732\3383108178.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()


In [4]:
export_columns = ['Date', 'ticker', 'NetProfitMargin']
# 导出到 CSV 文件
df_filtered_npm[export_columns].to_csv('NetProfitMargin.csv', index=False)

現金循環週期

In [12]:
import pandas as pd
import numpy as np

# Load the uploaded CSV file
file_path = '50合併.csv'
df = pd.read_csv(file_path)

# Convert Date column to datetime format and sort by date and ticker
df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')
df = df.sort_values(by=['ticker', 'Date'])

# Convert relevant columns to numeric types to avoid type errors
df['SALEQ'] = pd.to_numeric(df['SALEQ'], errors='coerce')
df['RECTQ'] = pd.to_numeric(df['RECTQ'], errors='coerce')
df['INVTQ'] = pd.to_numeric(df['INVTQ'], errors='coerce')
df['COGSQ'] = pd.to_numeric(df['COGSQ'], errors='coerce')
df['APQ'] = pd.to_numeric(df['APQ'], errors='coerce')

# Ensure each group has a continuous business day index, preserving individual date ranges for each ticker
df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()

# Calculate TTM Receivables Turnover (avg RECTQ / avg SALEQ)
df['Avg_RECTQ'] = df.groupby('ticker', group_keys=False)['RECTQ'].apply(lambda x: x.shift(1).rolling(window=252, min_periods=252).mean())
df['Avg_SALEQ'] = df.groupby('ticker', group_keys=False)['SALEQ'].apply(lambda x: x.shift(1).rolling(window=252, min_periods=252).mean())
df['Receivables_Days'] = (df['Avg_RECTQ'] / df['Avg_SALEQ']) * 365

# Calculate TTM Inventory Turnover (avg INVTQ / avg COGSQ)
df['Avg_INVTQ'] = df.groupby('ticker', group_keys=False)['INVTQ'].apply(lambda x: x.shift(1).rolling(window=252, min_periods=252).mean())
df['Avg_COGSQ'] = df.groupby('ticker', group_keys=False)['COGSQ'].apply(lambda x: x.shift(1).rolling(window=252, min_periods=252).mean())
df['Inventory_Days'] = (df['Avg_INVTQ'] / df['Avg_COGSQ']) * 365

# Calculate TTM Payables Turnover (avg APQ / avg COGSQ)
df['Avg_APQ'] = df.groupby('ticker', group_keys=False)['APQ'].apply(lambda x: x.shift(1).rolling(window=252, min_periods=252).mean())
df['Payables_Days'] = (df['Avg_APQ'] / df['Avg_COGSQ']) * 365

# Calculate Cash Conversion Cycle (CashCycle)
df['CashCycle'] = df['Receivables_Days'] + df['Inventory_Days'] - df['Payables_Days']

# Filter the DataFrame to remove rows with missing values in the calculated columns
df_filtered_ccc = df[['Date', 'ticker', 'SALEQ', 'RECTQ', 'INVTQ', 'COGSQ', 'APQ', 'Receivables_Days', 'Inventory_Days', 'Payables_Days', 'CashCycle']].dropna()

# Display the filtered results for Cash Conversion Cycle
df_filtered_ccc = df_filtered_ccc.sort_values(by=['ticker', 'Date'])
print(df_filtered_ccc)


C:\Users\asas9\AppData\Local\Temp\ipykernel_7732\599497380.py:6: DtypeWarning: Columns (2,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


             Date  ticker         SALEQ         RECTQ         INVTQ  \
269    2017-10-23  1101.0  2.471408e+10  2.065391e+10  9.360034e+09   
270    2017-10-24  1101.0  2.471408e+10  2.065391e+10  9.360034e+09   
271    2017-10-25  1101.0  2.471408e+10  2.065391e+10  9.360034e+09   
272    2017-10-26  1101.0  2.471408e+10  2.065391e+10  9.360034e+09   
273    2017-10-27  1101.0  2.471408e+10  2.065391e+10  9.360034e+09   
...           ...     ...           ...           ...           ...   
100266 2024-10-04  6505.0  1.706164e+11  5.608295e+10  9.504617e+10   
100267 2024-10-07  6505.0  1.706164e+11  5.608295e+10  9.504617e+10   
100268 2024-10-08  6505.0  1.706164e+11  5.608295e+10  9.504617e+10   
100269 2024-10-09  6505.0  1.706164e+11  5.608295e+10  9.504617e+10   
100271 2024-10-11  6505.0  1.706164e+11  5.608295e+10  9.504617e+10   

               COGSQ           APQ  Receivables_Days  Inventory_Days  \
269     1.944605e+10  7.466214e+09        295.884006      171.540391   
270

C:\Users\asas9\AppData\Local\Temp\ipykernel_7732\599497380.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()


In [14]:
export_columns = ['Date', 'ticker', 'CashCycle']
# 导出到 CSV 文件
df_filtered_ccc[export_columns].to_csv('CashCycle.csv', index=False)

普通股股數變動率

In [16]:
import pandas as pd
import numpy as np

# Load the uploaded CSV file
file_path = '50合併.csv'
df = pd.read_csv(file_path)

# Convert Date column to datetime format and sort by date and ticker
df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')
df = df.sort_values(by=['ticker', 'Date'])

# Convert relevant columns to numeric types to avoid type errors
df['CSHOQ'] = pd.to_numeric(df['CSHOQ'], errors='coerce')

# Ensure each group has a continuous business day index, preserving individual date ranges for each ticker
df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()

# Calculate Shares Outstanding Change (ShareChg) based on the past 4 quarters (approx. 252 trading days)
df['CSHOQ_1Y_ago'] = df.groupby('ticker', group_keys=False)['CSHOQ'].shift(252)
df['ShareChg'] = (df['CSHOQ'] / df['CSHOQ_1Y_ago']) - 1

# Filter the DataFrame to remove rows with missing values in the calculated columns
df_filtered_sharechg = df[['Date', 'ticker', 'CSHOQ', 'CSHOQ_1Y_ago', 'ShareChg']].dropna()

# Display the filtered results for Shares Outstanding Change
df_filtered_sharechg = df_filtered_sharechg.sort_values(by=['ticker', 'Date'])
print(df_filtered_sharechg)


C:\Users\asas9\AppData\Local\Temp\ipykernel_7732\4095078716.py:6: DtypeWarning: Columns (2,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


             Date  ticker         CSHOQ  CSHOQ_1Y_ago  ShareChg
269    2017-10-23  1101.0  3.692176e+09  3.692176e+09       0.0
270    2017-10-24  1101.0  3.692176e+09  3.692176e+09       0.0
271    2017-10-25  1101.0  3.692176e+09  3.692176e+09       0.0
272    2017-10-26  1101.0  3.692176e+09  3.692176e+09       0.0
273    2017-10-27  1101.0  3.692176e+09  3.692176e+09       0.0
...           ...     ...           ...           ...       ...
100266 2024-10-04  6505.0  9.525960e+09  9.525960e+09       0.0
100267 2024-10-07  6505.0  9.525960e+09  9.525960e+09       0.0
100268 2024-10-08  6505.0  9.525960e+09  9.525960e+09       0.0
100269 2024-10-09  6505.0  9.525960e+09  9.525960e+09       0.0
100271 2024-10-11  6505.0  9.525960e+09  9.525960e+09       0.0

[81456 rows x 5 columns]


C:\Users\asas9\AppData\Local\Temp\ipykernel_7732\4095078716.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()


In [18]:
export_columns = ['Date', 'ticker', 'ShareChg']
# 导出到 CSV 文件
df_filtered_sharechg[export_columns].to_csv('ShareChg.csv', index=False)

***資本獲取比率

In [22]:
import pandas as pd
import numpy as np

# Load the uploaded CSV file
file_path = '50合併.csv'
df = pd.read_csv(file_path)

# Convert Date column to datetime format and sort by date and ticker
df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')
df = df.sort_values(by=['ticker', 'Date'])

# Convert relevant columns to numeric types to avoid type errors
df['OANCF'] = pd.to_numeric(df['OANCF'], errors='coerce')
df['DVQ'] = pd.to_numeric(df['DVQ'], errors='coerce')
df['CAPXQ'] = pd.to_numeric(df['CAPXQ'], errors='coerce')

# Ensure each group has a continuous business day index, preserving individual date ranges for each ticker
df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()

# Calculate Capital Acquisition Ratio (CapAcqRatio) based on the past 4 quarters (approx. 252 trading days)
df['TTM_OANCFQ_DVQ'] = df.groupby('ticker', group_keys=False).apply(lambda x: (x['OANCF'] - x['DVQ']).shift(1).rolling(window=252, min_periods=252).sum())
df['TTM_CAPXQ'] = df.groupby('ticker', group_keys=False)['CAPXQ'].apply(lambda x: x.shift(1).rolling(window=252, min_periods=252).sum())
df['CapAcqRatio'] = df['TTM_OANCFQ_DVQ'] / df['TTM_CAPXQ']

# Filter the DataFrame to remove rows with missing values in the calculated columns
df_filtered_capacqratio = df[['Date', 'ticker', 'OANCF', 'DVQ', 'CAPXQ', 'TTM_OANCFQ_DVQ', 'TTM_CAPXQ', 'CapAcqRatio']].dropna()

# Display the filtered results for Capital Acquisition Ratio
df_filtered_capacqratio = df_filtered_capacqratio.sort_values(by=['ticker', 'Date'])
print(df_filtered_capacqratio)


C:\Users\asas9\AppData\Local\Temp\ipykernel_7732\4290710806.py:6: DtypeWarning: Columns (2,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


             Date  ticker         OANCF  DVQ         CAPXQ  TTM_OANCFQ_DVQ  \
269    2017-10-23  1101.0  3.208683e+09  0.0  4.983240e+08    8.473255e+11   
270    2017-10-24  1101.0  3.208683e+09  0.0  4.983240e+08    8.463709e+11   
271    2017-10-25  1101.0  3.208683e+09  0.0  4.983240e+08    8.454164e+11   
272    2017-10-26  1101.0  3.208683e+09  0.0  4.983240e+08    8.444618e+11   
273    2017-10-27  1101.0  3.208683e+09  0.0  4.983240e+08    8.435072e+11   
...           ...     ...           ...  ...           ...             ...   
100266 2024-10-04  6505.0 -9.599554e+09  0.0  1.572511e+09    1.728100e+12   
100267 2024-10-07  6505.0 -9.599554e+09  0.0  1.572511e+09    1.719867e+12   
100268 2024-10-08  6505.0 -9.599554e+09  0.0  1.572511e+09    1.711634e+12   
100269 2024-10-09  6505.0 -9.599554e+09  0.0  1.572511e+09    1.703401e+12   
100271 2024-10-11  6505.0 -9.599554e+09  0.0  1.572511e+09    1.695168e+12   

           TTM_CAPXQ  CapAcqRatio  
269     8.125352e+10    10.

C:\Users\asas9\AppData\Local\Temp\ipykernel_7732\4290710806.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.set_index('Date').groupby('ticker', group_keys=False).apply(lambda x: x.asfreq('B')).reset_index()
C:\Users\asas9\AppData\Local\Temp\ipykernel_7732\4290710806.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['TTM_OANCFQ_DVQ'] = df.groupby('ticker', group_keys=False).apply(lambda x: